In [1]:
import pandas as pd, numpy as np

In [3]:
rdat = pd.read_csv('../modelDatasets/RMdata88_99.csv')

In [4]:
rdat.head()

,Unnamed: 0,Unnamed: 0.1,UID,sYear,Rutting,IRI,PSC,Alli,Long,Is Bridge,...,meanMaxTemp,cumESALs,cumFTC,pAlli,pLong,IRIsi,esals,ppt,mint,maxt
0,0,11298,222,1988,0.236,8.815663,78,0.004356,0.023523,1,...,25.507500,547733.683099,0,1.980028,10.692152,1.669634,80705.085747,1653.949951,0.78,24.230000
1,1,11299,583,1988,0.512,14.358494,37,0.023864,0.011932,1,...,23.423749,308712.206196,1015,19.886370,9.943185,2.719413,44455.741252,2380.080078,-5.49,22.139999
2,2,11300,589,1988,0.512,14.358494,37,0.065909,0.032955,0,...,23.423749,297934.090065,1015,19.972458,9.986229,2.719413,42903.651207,2380.080078,-5.49,22.139999
3,3,11301,1623,1988,0.236,11.085579,62,0.011932,0.029924,0,...,30.033750,100146.413858,188,3.977274,9.974751,2.099542,14421.467543,184.009995,-8.78,28.590000
4,4,11302,1650,1988,0.236,12.669245,68,0.011932,0.029924,0,...,29.921250,78918.237150,188,5.423555,13.601933,2.399479,11364.528711,186.139999,-8.73,28.510000


In [5]:
uids = rdat.UID.unique()

In [6]:
print len(uids)

8861


In [7]:
cds = pd.read_csv('../FinalDatasets/RCSegCoord.csv')

In [8]:
cds.head()

,Unnamed: 0,UID,Longitude,Latitude
0,0,222,-121.802218,47.860322
1,1,540,-121.252570,47.715980
2,2,2431,-118.301093,47.646535
3,3,704,-121.000382,47.774584
4,4,776,-120.868954,47.787672


In [9]:
print len(rdat[rdat.UID.isin(cds.UID)]) == len(rdat)

True


In [10]:
print len(cds) == len(cds.UID.unique())

True


In [13]:
cds.UID.dtype

dtype('int64')

In [12]:
cds.UID = cds.UID.astype('int')

In [14]:
cds.to_csv('../FinalDatasets/RCSegCoord.csv')

In [16]:
mds = pd.read_csv('../FinalDatasets/RMid2icao.csv') # contains match data for UID and ICAO codes to obtain FTCycle data

In [17]:
mds.head()

,InputID,TargetID,Distance
0,222,KAWO,0.469535
1,540,KESW,0.465864
2,2431,KSKA,0.657846
3,704,KESW,0.552878
4,776,KESW,0.621369


In [18]:
mds.index = mds.InputID

In [19]:
def getCode(row):
    index = row['UID']
    return mds.loc[index,'TargetID']

In [21]:
rdat.UID = rdat.UID.astype('int')

In [22]:
rdat.loc[:, 'ICAO'] = rdat.apply(getCode, axis =1)

Loading the weather dataset with all the variables. Variable of interest is the freeze thaw cycles..

In [23]:
wds = pd.read_excel('../DataFiles/AnnualWeatherR.xlsx') #contains yrs 1981 -1999
wd = wds[['Year', 'FTCycles', 'Airport']]
ft = wd.pivot('Airport', 'Year', 'FTCycles')

In [24]:
ft.head()

Year,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
Airport,,,,,,,,,,,,,,,,,,,,
KALW,0,12,30,25,23,48,26,23,22,28,25,17,19,38,21,18,30,11,11,11
KAWO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
KBFI,8,3,8,5,9,16,4,5,0,8,18,3,9,10,7,3,14,0,7,2
KBLI,13,4,12,5,11,16,9,8,9,14,24,10,7,7,7,7,27,3,9,10
KCLM,0,2,1,1,7,7,6,5,5,7,18,7,5,8,4,2,20,1,6,7


In [28]:
print ft.columns.dtype
print rdat.sYear.dtype

int64
int64


In [29]:
def getFTC(row):
    index = row['ICAO']
    year  = row['sYear']
    return ft.loc[index, year]

In [30]:
rdat.loc[:, 'ftc'] = rdat.apply(getFTC, axis = 1)

In [31]:
rdat.head()

,Unnamed: 0,Unnamed: 0.1,UID,sYear,Rutting,IRI,PSC,Alli,Long,Is Bridge,...,cumFTC,pAlli,pLong,IRIsi,esals,ppt,mint,maxt,ICAO,ftc
0,0,11298,222,1988,0.236,8.815663,78,0.004356,0.023523,1,...,0,1.980028,10.692152,1.669634,80705.085747,1653.949951,0.78,24.230000,KAWO,0
1,1,11299,583,1988,0.512,14.358494,37,0.023864,0.011932,1,...,1015,19.886370,9.943185,2.719413,44455.741252,2380.080078,-5.49,22.139999,KESW,109
2,2,11300,589,1988,0.512,14.358494,37,0.065909,0.032955,0,...,1015,19.972458,9.986229,2.719413,42903.651207,2380.080078,-5.49,22.139999,KESW,109
3,3,11301,1623,1988,0.236,11.085579,62,0.011932,0.029924,0,...,188,3.977274,9.974751,2.099542,14421.467543,184.009995,-8.78,28.590000,KEPH,19
4,4,11302,1650,1988,0.236,12.669245,68,0.011932,0.029924,0,...,188,5.423555,13.601933,2.399479,11364.528711,186.139999,-8.73,28.510000,KEPH,19


In [32]:
rdat.columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'UID', u'sYear', u'Rutting', u'IRI',
       u'PSC', u'Alli', u'Long', u'Is Bridge', u'Lane Miles',
       u'Segment Lane Miles', u'ESALs Since Major Rehab', u'sfThick',
       u'Tthick', u'TSfThick', u'STcode', u'cumPPT', u'meanMinTemp',
       u'meanMaxTemp', u'cumESALs', u'cumFTC', u'pAlli', u'pLong', u'IRIsi',
       u'esals', u'ppt', u'mint', u'maxt', u'ICAO', u'ftc'],
      dtype='object')

In [33]:
fmd = rdat[[u'UID', u'sYear', u'Rutting', u'IRI', u'PSC', u'Alli', u'Long', u'Is Bridge', u'Lane Miles',
       u'Segment Lane Miles', u'ESALs Since Major Rehab', u'sfThick',
       u'Tthick', u'TSfThick', u'STcode', u'cumPPT', u'meanMinTemp',
       u'meanMaxTemp', u'cumESALs', u'cumFTC', u'pAlli', u'pLong', u'IRIsi',
       u'esals', u'ppt', u'mint', u'maxt', u'ICAO', u'ftc']]

In [34]:
fmd.to_csv('../modelDatasets/rmData1115.csv')